# Test email

In [2]:
import pandas as pd

S3_FILE_PATH = "s3://civis-test-public/"
US_COUNTY_URL = f"{S3_FILE_PATH}us-county-time-series.parquet"

In [3]:
df = pd.read_parquet(US_COUNTY_URL)

In [4]:
display(df.head())

,county,state,fips,date,Lat,Lon,cases,deaths,incident_rate,people_tested,state_cases,state_deaths,new_cases,new_deaths,new_state_cases,new_state_deaths
62,Autauga,Alabama,01001,2020-03-24 07:00:00+00:00,32.539527,-86.644082,1,0,1.789901,<NA>,298,0,1,0,74,0
63,Autauga,Alabama,01001,2020-03-25 07:00:00+00:00,32.539527,-86.644082,5,0,8.949507,<NA>,472,1,4,0,174,1
64,Autauga,Alabama,01001,2020-03-26 07:00:00+00:00,32.539527,-86.644082,6,0,10.739408,<NA>,574,1,1,0,102,0
65,Autauga,Alabama,01001,2020-03-27 07:00:00+00:00,32.539527,-86.644082,6,0,10.739408,<NA>,684,4,0,0,110,3
66,Autauga,Alabama,01001,2020-03-28 07:00:00+00:00,32.539527,-86.644082,6,0,10.739408,<NA>,801,4,0,0,117,0
